In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import zipfile
import unicodedata
import string
import random
import time
import math
import numpy as np
import mxnet as mx
import os


In [2]:
data_dir_name='./char-rnn-data'

In [3]:
def download_data(data_dir_name):
    fname = mx.test_utils.download(url='https://download.pytorch.org/tutorial/data.zip', dirname=data_dir_name, overwrite=False)
    print (fname)
    zip = zipfile.ZipFile(fname)
    zip.extractall(data_dir_name)
    zip.close()

download_data(data_dir_name)

./char-rnn-data/data.zip


In [4]:
def findFiles(path): return glob.glob(path)

all_letters = string.ascii_letters + " .,;'"
vocab_size = len(all_letters)

# Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []
total_lines = 0

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles(data_dir_name + '/data/names/*.txt'):
    category = filename.split('/')[-1].split('.')[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines
    total_lines += len(lines)

n_categories = len(all_categories)

print('Language Categories:', all_categories)

Language Categories: [u'Portuguese', u'Scottish', u'English', u'Vietnamese', u'Japanese', u'German', u'Polish', u'Spanish', u'Korean', u'French', u'Russian', u'Arabic', u'Dutch', u'Irish', u'Greek', u'Czech', u'Chinese', u'Italian']


In [9]:
with open(data_dir_name + '/data123.txt', 'w') as d:
    print (d.name)
    for category in all_categories:
        for line in category_lines[category]:
            d.write(line)
            d.write(',')
            d.write(category)
            d.write('\n')

./char-rnn-data/data123.txt


In [33]:
val_data ={}
for key in category_lines:
    val_data[key] = category_lines[key][:int(0.10 * len(category_lines[key]))]
    del category_lines[key][:int(0.10 * len(category_lines[key]))]

In [34]:
num_l=0
num_v=0
for key in category_lines:
    num_l += len(category_lines[key])
    num_v += len(val_data[key])

print(num_l)
print(num_v)

18076
1998


In [84]:
with open(data_dir_name + '/train_data.txt', 'w') as d, open(data_dir_name + '/train_label.txt', 'w') as l, open(data_dir_name + '/val_data.txt', 'w') as vd, open(data_dir_name + '/val_label.txt', 'w') as vl:
    for category in all_categories:
        for line in category_lines[category]:
            d.write(line)
            d.write('\n')
            l.write(category)
            l.write('\n')
        for line in val_data[category]:
            vd.write(line)
            vd.write('\n')
            vl.write(category)
            vl.write('\n')

In [51]:
with zipfile.ZipFile(data_dir_name+'.zip', 'w', zipfile.ZIP_DEFLATED) as ziph:
    # ziph is zipfile handle
    for root, dirs, files in os.walk(data_dir_name):
        for file in files:
            ziph.write(os.path.join(root, file))


In [3]:
def download_data(data_dir_name):
    fname = mx.test_utils.download(url='http://github.com/nswamy/web-data/blob/char_rnn/mxnet/doc/tutorials/rnn/char-rnn-data.zip', dirname=data_dir_name, overwrite=True)
    print (fname)
#     zip = zipfile.ZipFile(fname)
#     zip.extractall(data_dir_name)
#     zip.close()

download_data('zz')

zz/char-rnn-data.zip


In [85]:
def read_input(path):
    with open(path + '/train_data.txt', 'r') as td, open(path + '/train_label.txt', 'r') as tl:
        train_data = [line.strip('\n') for line in td.readlines()]
        train_label = [line.strip('\n') for line in tl.readlines()]

    with open(path + '/val_data.txt', 'r') as vd, open(path + '/val_label.txt', 'r') as vl:
        eval_data = [line.strip('\n') for line in vd.readlines()]
        eval_label = [line.strip('\n') for line in vl.readlines()]

    return train_data, train_label, eval_data, eval_label

train_data, train_label, eval_data, eval_label = read_input(data_dir_name)

In [91]:
vocab = string.ascii_letters + " .,;'"
train_labels_unique = list(set(train_label))
eval_labels_unique = list(set(eval_label))

In [118]:
eval_label_onehot = mx.nd.one_hot(mx.nd.array([eval_labels_unique.index(label) for label in eval_label]), len(eval_labels_unique))
train_label_onehot = mx.nd.one_hot(mx.nd.array([train_labels_unique.index(label) for label in train_label]), len(train_labels_unique))

In [119]:
train_data_onehot = [mx.nd.one_hot(mx.nd.array([vocab.find(ch) for ch in line]), vocab_size) for line in train_data]
eval_data_onehot = [mx.nd.one_hot(mx.nd.array([vocab.find(ch) for ch in line]), vocab_size) for line in eval_data]

In [120]:
mx.io.NDArrayIter?